## **Data Redundancy and Distributed Computing Workshop - TD3-A**  
In this notebook we explore the transition from local computation to decentralized prediction models. We will develop multiple predictive models on a selected dataset, integrate them into an API, and introduce a **consensus mechanism** with a **slashing protocol** to enhance reliability in a decentralized setting.  

_Authors_: Alessia SARRITZU, Alberto MARTINELLI

### Dataset Selection 
For this workshop, we decided to work with the **Titanic dataset**. It provides information about passengers aboard the Titanic, including their demographics, ticket class, and whether they survived the disaster.  

**Core Features**
- **Survived (`survived`)** – Target variable indicating whether the passenger survived (1 = Yes, 0 = No).  
- **Passenger Class (`pclass`)** – The ticket class (1 = First, 2 = Second, 3 = Third).  
- **Sex (`sex`)** – Gender of the passenger (`male` or `female`).  
- **Age (`age`)** – Age of the passenger (may contain missing values).  
- **Siblings/Spouses aboard (`sibsp`)** – Number of siblings/spouses traveling with the passenger.  
- **Parents/Children aboard (`parch`)** – Number of parents/children traveling with the passenger.  
- **Fare (`fare`)** – The fare paid for the ticket.  
- **Embarked (`embarked`)** – Port of embarkation (`C` = Cherbourg, `Q` = Queenstown, `S` = Southampton).  

**Additional Features**
- **Class (`class`)** – Alternative representation of `pclass` (`First`, `Second`, `Third`).  
- **Who (`who`)** – Categorizes passengers as `man`, `woman`, or `child` based on age and gender.  
- **Adult Male (`adult_male`)** – Boolean flag (`True` = adult male, `False` = otherwise).  
- **Deck (`deck`)** – Deck location of the cabin (many missing values).  
- **Embark Town (`embark_town`)** – Full name of the embarkation town (`Southampton`, `Cherbourg`, `Queenstown`).  
- **Alive (`alive`)** – String representation of survival status (`yes` or `no`).  
- **Alone (`alone`)** – Boolean indicating if the passenger was traveling alone (`True` = alone, `False` = had family aboard).  

Next, we will load and preprocess the dataset before developing individual predictive models.

### Data cleaning and Preprocessing

Before training our machine learning models, it is essential to perform **data cleaning and preprocessing** to ensure that the dataset is free of inconsistencies, missing values, and irrelevant features. This step enhances model performance and ensures reliable predictions.  

In this section, we:  
1. **Load the Titanic dataset** and examine its structure.  
2. **Analyze the class distribution** to understand the balance between survivors and non-survivors.  
3. **Handle missing values**, specifically by dropping the `deck` column (which has excessive missing data) and removing any remaining incomplete rows.  
4. **Prepare features for machine learning**, including encoding categorical variables (`sex`, `pclass`, and `embarked`) and standardizing numerical features (`age`, `sibsp`, `parch`, `fare`).  
5. **Split the dataset into training and testing sets**, ensuring our models are evaluated fairly.  

In [58]:
import pandas as pd
import seaborn as sns

df = sns.load_dataset('titanic')

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB
None


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [59]:
class_distribution = df['survived'].value_counts()
print(class_distribution)

survived
0    549
1    342
Name: count, dtype: int64


In [60]:
missing_values = df.isnull().sum()
print(f"Missing Values before cleaning: \n{missing_values}\n")

# Remove 'deck' column since it has many missing values
df.drop(columns=['deck'], inplace=True)

# Remove rows with any missing values
df.dropna(inplace=True)

# Check new class distribution
class_distribution = df['survived'].value_counts()
print(class_distribution)

df.head()

Missing Values before cleaning: 
survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

survived
0    424
1    288
Name: count, dtype: int64


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True


In [61]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Select features and target variable
X = df[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]
y = df['survived']

# Define preprocessing pipeline
scaler = StandardScaler()
preprocessor = ColumnTransformer([
    ('num', scaler, ['age', 'sibsp', 'parch', 'fare']),
    ('cat', OneHotEncoder(drop='first'), ['pclass', 'sex', 'embarked'])
])

# Split dataset into training & test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply preprocessing to training & test data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

## 1. Practical Exercise: From Local to Decentralized Computation

### **Q1** - Model Development and Deployment

In this section, we will focus on building, evaluating, and deploying predictive models for the selected dataset. The key objectives are:

- **Develop predictive models** : _Logistic Regression_, _Random Forest_ and _SVM_.
- **Evaluate model accuracy and performance**: precision, recall, f1-score, accuracy.  
- **Adapt the models for API access**: expose them via a Flask application with a GET `/predict` endpoint (`app.py`).
- **Standardize the API response format**: ensure consistency across all models for seamless integration.
```json
    {
        "model": "<model_name>",
        "input_features": {
            "pclass": 3,
            "sex": "male",
            "age": 22,
            "sibsp": 1,
            "parch": 0,
            "fare": 7.25,
            "embarked": "S"
        },
        "prediction": "<Survived | Did not survive>"
    }
```

Additionally, the models deployed using Flask (`app.py`) can be tested through (`test.http`, section _Q1_). 

In [62]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Train logistic regression model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Predict & evaluate
y_pred_logreg = logreg.predict(X_test)
accuracy_logreg = accuracy_score(y_test, y_pred_logreg)
print(f'Logistic Regression Classification Report:\n{classification_report(y_test, y_pred_logreg)}')

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.90      0.84        80
           1       0.84      0.68      0.75        63

    accuracy                           0.80       143
   macro avg       0.81      0.79      0.80       143
weighted avg       0.81      0.80      0.80       143



In [63]:
from sklearn.ensemble import RandomForestClassifier

# Train Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict & evaluate
y_pred_rf = rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f'Random Forest Classification Report:\n{classification_report(y_test, y_pred_rf)}')

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.84      0.81        80
           1       0.78      0.71      0.74        63

    accuracy                           0.78       143
   macro avg       0.78      0.78      0.78       143
weighted avg       0.78      0.78      0.78       143



In [64]:
from sklearn.svm import SVC

# Train SVM model
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

# Predict & evaluate
y_pred_svm = svm.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f'SVM Classification Report:\n{classification_report(y_test, y_pred_svm)}')

SVM Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.85      0.79        80
           1       0.76      0.62      0.68        63

    accuracy                           0.75       143
   macro avg       0.75      0.73      0.74       143
weighted avg       0.75      0.75      0.74       143



In [65]:
import joblib

# Save models & preprocessor
joblib.dump(logreg, "data/logreg_model.pkl")
joblib.dump(rf, "data/rf_model.pkl")
joblib.dump(svm, "data/svm_model.pkl")
joblib.dump(preprocessor, "data/preprocessor.pkl")

['data/preprocessor.pkl']

### **Q2** - Consensus Model Development

In this section, we will focus on implementing a **consensus-based prediction model** that aggregates the outputs from multiple individual models to enhance reliability and robustness. The key objectives are:

- **Develop a consensus model** by averaging predictions from _Logistic Regression_, _Random Forest_, and _SVM_.  
- **Expose the consensus model via API** using a Flask `/predict/consensus` endpoint (`app.py`).  
- **Evaluate performance** of the consensus model.
- **Ensure inter-computer connectivity** using **ngrok**, allowing external systems to access the prediction service.

<div style="display: flex; justify-content: center; align-items: center;">
    <img src="../images/ngrok-activation.png" alt="NGROK Activation" width="500">
    <img src="../images/ngrok-consensus.png" alt="NGROK Consensus" width="500">
</div>
<br>  

To maintain consistency with individual model predictions, the **consensus model API** response format follows the same structured response:
```json
{
    "model": "consensus",
    "input_features": {
        "pclass": 3,
        "sex": "male",
        "age": 22,
        "sibsp": 1,
        "parch": 0,
        "fare": 7.25,
        "embarked": "S"
    },
    "individual_predictions": {
        "logistic_regression": "Did not survive",
        "random_forest": "Survived",
        "svm": "Did not survive"
    },
    "final_prediction": "<Survived | Did not survive>"
}
```
The **final consensus prediction** is determined by averaging the outputs of all models and rounding the result. <br>
The model can be tested through (`test.http`, section _Q2_) or through **ngrok** if the local server is running. 

In [66]:
from sklearn.metrics import accuracy_score
import numpy as np

# Get predictions for the test set using all models
individual_preds = {model: model.predict(X_test) for model in [logreg, rf, svm]}

# Compute consensus predictions
consensus_preds = np.round(np.mean(np.array(list(individual_preds.values())), axis=0))

# Evaluate accuracy
consensus_accuracy = accuracy_score(y_test, consensus_preds)
print(f"Consensus Model Classification Report:\n{classification_report(y_test,consensus_preds)}")


Consensus Model Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.90      0.83        80
           1       0.84      0.65      0.73        63

    accuracy                           0.79       143
   macro avg       0.80      0.78      0.78       143
weighted avg       0.80      0.79      0.79       143



## 2. Introducing Consensus with Slashing Mechanism

### **Q3** - Weighted Consensus Model

In this section, we enhance the consensus model by introducing a **dynamic weighting** system to refine predictions based on each model's performance over time. Instead of treating all models equally, we adjust their influence on the final prediction using dynamically updated weights. <br>
The key objectives are:

- Implement a **weighting system** where model contributions are adjusted based on accuracy.
- Weights range from 0 to 1, reflecting each **model's reliability** in past predictions.
- Weights are updated per **batch**, refining consensus predictions as more data is processed.

The model can be tested through (`test.http`, section _Q3_) or through **ngrok** if the local server is running. 

In [67]:
import numpy as np
import json
from sklearn.metrics import accuracy_score, classification_report

# Compute initial model accuracies
model_accuracies = {
    "logistic_regression": accuracy_score(y_test, y_pred_logreg),
    "random_forest": accuracy_score(y_test, y_pred_rf),
    "svm": accuracy_score(y_test, y_pred_svm)
}

# Normalize initial weights to sum to 1
weights = np.array(list(model_accuracies.values()))
weights /= weights.sum()  

# Function to dynamically update weights
def update_weights(y_true, predictions, weights, learning_rate=0.1):
    predictions = np.array(predictions)  
    y_true = np.array(y_true).flatten() 

    # Compute per-model accuracy in this batch
    correct_predictions = (predictions.T == y_true).T  
    accuracy_adjustments = correct_predictions.mean(axis=0)  
    # Apply weighted adjustment
    weights = weights * (1 + learning_rate * (accuracy_adjustments - weights))

    # Normalize weights to sum to 1
    weights /= weights.sum()

    return weights

# Simulate multiple batches of predictions
batch_size = 10
num_batches = len(y_test) // batch_size
pred_array = np.column_stack([preds.reshape(-1, 1) for preds in individual_preds.values()])

for i in range(num_batches):
    batch_start = i * batch_size
    batch_end = batch_start + batch_size

    # Get batch data
    batch_y = y_test.iloc[batch_start:batch_end].values  
    batch_preds = pred_array[batch_start:batch_end]  

    # Compute weighted consensus prediction
    weighted_preds = np.round(np.average(batch_preds, axis=1, weights=weights)).astype(int)

    # Update weights based on batch performance
    weights = update_weights(batch_y, batch_preds, weights)


weights_dict = dict(zip(["logistic_regression", "random_forest", "svm"], weights.tolist()))
with open("data/model_weights.json", "w") as f:
    json.dump(weights_dict, f)

print(f"\nFinal Weights saved to data/model_weights.json: {weights_dict}")
print(f"\nFinal Weighted Consensus Classification Report:\n{classification_report(y_test, np.round(np.average(pred_array, axis=1, weights=weights)).astype(int))}")



Final Weights saved to data/model_weights.json: {'logistic_regression': 0.34943741580559107, 'random_forest': 0.33707891990152916, 'svm': 0.31348366429287977}

Final Weighted Consensus Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.90      0.83        80
           1       0.84      0.65      0.73        63

    accuracy                           0.79       143
   macro avg       0.80      0.78      0.78       143
weighted avg       0.80      0.79      0.79       143



### **Q4** - Proof-of-Stake Consensus & Slashing Mechanism

In this section, we develop a **proof-of-stake (PoS)** consensus mechanism combined with a slashing protocol to ensure accountability and trust in the decentralized prediction system. Each model stakes an initial deposit (1000 euros) upon registration to participate in the prediction network. This deposit serves as a security measure, ensuring that models contribute accurate and reliable predictions.
The key objectives are:

- **Stake-Based Participation**: Models must provide an initial deposit to be eligible for participation.
- **Penalty (Slashing) System**: Models that consistently make inaccurate predictions will have their stake reduced.
- **Performance-Based Adjustments**: Weights are adjusted not only based on accuracy but also on penalties for unreliable models.
- **Encouraging Trustworthiness**: Reliable models maintain or grow their stake, while inaccurate models risk financial loss.

This system discourages malicious or low-quality contributions and promotes accuracy-driven participation. The model balance and slashing mechanism will are tracked in a local JSON database to simplify implementation.

The model can be tested through (`test.http`, section Q4) or through **ngrok** if the local server is running.

#### **Q4.1 Initialize & Load Balances**

In [68]:
import json
import os
import numpy as np
from sklearn.metrics import accuracy_score

BALANCE_FILE = "data/model_balances.json"
INITIAL_BALANCE = 1000  # Initial deposit for each model
SLASH_PENALTY = 50  # Penalty for poor performance (slashing)
SLASHING_THRESHOLD = 0.3  # Accuracy threshold for slashing
models = ["logistic_regression", "random_forest", "svm"]

def initialize_and_save_balances(y_test, individual_preds):
    """
    Initializes model balances, updates them based on accuracy, and saves to JSON.
    
    - Models start with an initial deposit.
    - If accuracy is below the threshold, a slashing penalty is applied.
    - Balances are saved to 'data/model_balances.json'.
    """
    # Load existing balances if they exist, otherwise initialize them
    if os.path.exists(BALANCE_FILE):
        with open(BALANCE_FILE, "r") as f:
            balances = json.load(f)
    else:
        balances = {model: INITIAL_BALANCE for model in models}

    # Apply slashing for models below accuracy threshold
    for model_name, accuracy in model_accuracies.items():
        if accuracy < SLASHING_THRESHOLD:
            balances[model_name] = max(0, balances[model_name] - SLASH_PENALTY)  # Ensure balance doesn't go negative

    # Save updated balances to JSON
    with open(BALANCE_FILE, "w") as f:
        json.dump(balances, f)

    print(f"\nModel balances initialized and saved: {balances}")

# Call function to initialize and save balances
initialize_and_save_balances(y_test, individual_preds)



Model balances initialized and saved: {'logistic_regression': 1000, 'random_forest': 1000, 'svm': 1000}


#### **Q4.2 Load Balances for Use in Weighted Consensus**

In [69]:
def load_balances():
    """Load model balances from JSON file or initialize if missing."""
    if os.path.exists(BALANCE_FILE):
        with open(BALANCE_FILE, "r") as f:
            return json.load(f)
    else:
        # If missing, reinitialize
        return {model: INITIAL_BALANCE for model in models}


#### **Q4.3 Weighted Consensus with Proof-of-Stake**

In [70]:
def update_weights_with_slashing(y_true, predictions, weights, learning_rate=0.1):
    """
    Adjust weights dynamically after each batch based on accuracy.
    Apply slashing if a model's accuracy falls below a threshold.
    """
    predictions = np.array(predictions)  # Shape: (batch_size, num_models)
    y_true = np.array(y_true).flatten()  # Shape: (batch_size,)

    correct_predictions = (predictions.T == y_true).T
    accuracy_adjustments = correct_predictions.mean(axis=0)  # Compute accuracy per model

    # Load current balances
    balances = load_balances()

    # Adjust weights using a combination of performance and stake
    for i, model_name in enumerate(models):
        if accuracy_adjustments[i] < SLASHING_THRESHOLD:
            balances[model_name] = max(0, balances[model_name] - SLASH_PENALTY)  # Apply slashing

        # Use stake as a weight factor
        weights[i] = balances[model_name] * (1 + learning_rate * (accuracy_adjustments[i] - weights[i]))

    # Normalize weights
    weights /= weights.sum()

    # Save updated balances
    with open(BALANCE_FILE, "w") as f:
        json.dump(balances, f)

    return weights

#### **Q4.4 Apply PoS Consensus Over Multiple Batches**

In [71]:
# Simulate multiple batches of predictions
batch_size = 10
num_batches = len(y_test) // batch_size
pred_array = np.column_stack([
    preds.reshape(-1, 1) for preds in individual_preds.values()
])

for i in range(num_batches):
    batch_start = i * batch_size
    batch_end = batch_start + batch_size

    # Get batch data
    batch_y = y_test.iloc[batch_start:batch_end].values  # Convert to NumPy array
    batch_preds = pred_array[batch_start:batch_end]  # Shape: (batch_size, num_models)

    # Compute proof-of-stake weighted consensus prediction
    balances = load_balances()
    pos_weights = np.array([balances[m] for m in models], dtype=float)  # Use stake as weight and ensure float type
    pos_weights /= pos_weights.sum()  # Normalize

    weighted_preds = np.round(np.average(batch_preds, axis=1, weights=pos_weights)).astype(int)

    # Update weights with slashing
    pos_weights = update_weights_with_slashing(batch_y, batch_preds, pos_weights)

    print(f"\nBatch {i+1}: Updated Weights = {pos_weights}")


Batch 1: Updated Weights = [0.33438819 0.33122363 0.33438819]

Batch 2: Updated Weights = [0.33651805 0.33014862 0.33333333]

Batch 3: Updated Weights = [0.33438819 0.33438819 0.33122363]

Batch 4: Updated Weights = [0.3343983  0.33120341 0.3343983 ]

Batch 5: Updated Weights = [0.33654877 0.33333333 0.3301179 ]

Batch 6: Updated Weights = [0.3344086 0.3344086 0.3311828]

Batch 7: Updated Weights = [0.33017876 0.33648791 0.33333333]

Batch 8: Updated Weights = [0.33333333 0.33333333 0.33333333]

Batch 9: Updated Weights = [0.33226496 0.33547009 0.33226496]

Batch 10: Updated Weights = [0.33438819 0.33438819 0.33122363]

Batch 11: Updated Weights = [0.33333333 0.33648791 0.33017876]

Batch 12: Updated Weights = [0.33658009 0.33008658 0.33333333]

Batch 13: Updated Weights = [0.33547009 0.33226496 0.33226496]

Batch 14: Updated Weights = [0.33227513 0.33544974 0.33227513]


#### **Q4.5 Evaluate Accuracy and Performances**

In [73]:
weights_dict = dict(zip(["logistic_regression", "random_forest", "svm"], pos_weights.tolist()))
with open("data/pos_model_weights.json", "w") as f:
    json.dump(weights_dict, f)

print(f"\nFinal Weights saved to data/model_weights.json: {weights_dict}")
print(f"\nFinal Weighted PoS Consensus Classification Report:\n{classification_report(y_test, np.round(np.average(pred_array, axis=1, weights=final_pos_weights)).astype(int))}")

[0.33227513 0.33544974 0.33227513]

Final Weights saved to data/model_weights.json: {'logistic_regression': 0.3322751322751323, 'random_forest': 0.3354497354497355, 'svm': 0.3322751322751323}

Final Weighted PoS Consensus Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.90      0.83        80
           1       0.84      0.65      0.73        63

    accuracy                           0.79       143
   macro avg       0.80      0.78      0.78       143
weighted avg       0.80      0.79      0.79       143

